In [44]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct

class QdrantStorage:
    def __init__(self, url="http://localhost:200", collection="docs", dim=3072):
        self.client = QdrantClient(url=url, timeout=30)
        self.collection = collection
        if not self.client.collection_exists(self.collection):
            self.client.create_collection(
                collection_name=self.collection,
                vectors_config=VectorParams(size=dim, distance=Distance.COSINE),
            )
            
    
    def upsert(self, ids, vectors, payloads):
        points = [
            PointStruct(id=ids[i], vector=vectors[i], payload=payloads[i]) 
            for i in range(len(ids))
        ]
        self.client.upsert(self.collection, points=points)
    
    def search(self, query_vector, top_k: int = 5):
        results = self.client.query_points(
            collection_name=self.collection,
            query=query_vector,
            limit=top_k,
            with_payload=True,
        )
        
        contexts = []
        sources = set()
        
        # QueryResponse.points içindeki ScoredPoint objelerine eriş
        for point in results.points:
            text = point.payload.get("text", "")
            source = point.payload.get("source", "")
            if text:
                contexts.append(text)
                sources.add(source)
        
        return {"contexts": contexts, "sources": list(sources)}

In [45]:
# --- embedding client (OpenRouter) ---
client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="</>",
)

def embed_texts(texts: list[str]) -> list[list[float]]:
    # Note: embedding model must match Qdrant vector dim
    res = client.embeddings.create(
        model="openai/text-embedding-3-large",
        input=texts,
        # encoding_format="float",
        extra_headers={
            "HTTP-Referer": "farukseker.com.tr",
            "X-Title": "farukseker",
        },
    )
    return [d.embedding for d in res.data]

In [48]:
from os import getenv
from uuid import uuid4
from openai import OpenAI

# --- test documents ---
TEST_DOCS = [
    {"text": "cat is a vector database for similarity search.", "source": "doc1"},
    {"text": "RAG systems combine retrieval with generation.", "source": "doc2"},
    {"text": "Embeddings turn text into numerical vectors.", "source": "doc3"},
]





# --- load test docs into qdrant ---
storage = QdrantStorage(collection="docs", dim=3072)

texts = [d["text"] for d in TEST_DOCS]
vectors = embed_texts(texts)

ids = [str(uuid4()) for _ in TEST_DOCS]
payloads = TEST_DOCS

storage.upsert(ids=ids, vectors=vectors, payloads=payloads)

# --- test search ---
query = "Qdrant"
query_vector = embed_texts([query])[0]

result = storage.search(query_vector, top_k=3)
print(result)


{'contexts': ['RAG systems combine retrieval with generation.', 'cat is a vector database for similarity search.', 'Embeddings turn text into numerical vectors.'], 'sources': ['doc1', 'doc2', 'doc3']}


In [47]:
# Upsert sonrası koleksiyonu kontrol et
storage.upsert(ids=ids, vectors=vectors, payloads=payloads)

# Koleksiyon bilgisi
try:
    info = storage.client.get_collection(collection_name="docs")
    print(f"✓ Koleksiyondaki nokta sayısı: {info.points_count}")
except Exception as e:
    print(f"✗ Koleksiyon bilgisi alınamadı: {e}")

# Manuel scroll ile kontrol
try:
    scroll_result = storage.client.scroll(collection_name="docs", limit=10)
    print(f"✓ Scroll sonucu: {len(scroll_result[0])} nokta bulundu")
    print(f"✓ İlk nokta: {scroll_result[0][0] if scroll_result[0] else 'YOK'}")
except Exception as e:
    print(f"✗ Scroll hatası: {e}")

# Şimdi arama yap
result = storage.search(query_vector, top_k=3)
print(f"\n📊 Final Result: {result}")

✓ Koleksiyondaki nokta sayısı: 51
✓ Scroll sonucu: 10 nokta bulundu
✓ İlk nokta: id='036f9b34-38b2-4b92-8fdc-5221cb20ade9' payload={'text': 'What is RAG system ?', 'source': 'doc3'} vector=None shard_key=None order_value=None

📊 Final Result: {'contexts': ['Qdrant is a vector database for similarity search.', 'Qdrant is a vector database for similarity search.', 'Qdrant is a vector database for similarity search.'], 'sources': ['doc1']}
